In [29]:
# This should be roughly the content of the first code cell
import numpy as np
import random
np.random.seed(1337)
random.seed(1337)


## Implement ID3 decision-tree inference algorithm from scratch    

In [30]:
import pandas as pd
#read election_clean.csv
df1 = pd.read_csv("elections_clean.csv")

In [31]:
# select categorical features from dataframe
df = df1[['democrat', 'county', 'state', 'education', 'religion', 'ethnic_male','ethnic_female']]
df

,democrat,county,state,education,religion,ethnic_male,ethnic_female
0,0,Sioux County,NE,Educated,CHRISTIAN GENERIC,MULTI MALE RATE,NATIVE AMERICAN FEMALE RATE
1,1,Baltimore city,MD,Educated,OTHER MISC,WHITE MALE RATE,WHITE FEMALE RATE
2,1,Chittenden County,VT,Educated,MAINLINE CHRISTIAN,ASIAN MALE RATE,BLACK FEMALE RATE
3,0,Prince of Wales-Hyder Census Area,AK,Uneducated,OTHER MISC,BLACK MALE RATE,BLACK FEMALE RATE
4,0,St. Johns County,FL,Educated,MAINLINE CHRISTIAN,BLACK MALE RATE,BLACK FEMALE RATE
...,...,...,...,...,...,...,...
3140,0,Clinton County,MO,Educated,CATHOLIC,MULTI MALE RATE,BLACK FEMALE RATE
3141,0,Union County,AR,Uneducated,MAINLINE CHRISTIAN,BLACK MALE RATE,BLACK FEMALE RATE
3142,0,Garfield County,NE,Educated,MAINLINE CHRISTIAN,MULTI MALE RATE,BLACK FEMALE RATE
3143,0,Greene County,IN,Uneducated,MAINLINE CHRISTIAN,MULTI MALE RATE,NATIVE AMERICAN FEMALE RATE


    When I ran my prediction function, the program reported an error.
    I found out after testing that it was because some samples had missing values. 
    So I use dropna() here to remove the samples with missing values.

In [32]:
#cleanning the data
df.dropna(inplace = True)

/var/folders/dz/ffb7x_vj5vl9758_4vqfvj3m0000gn/T/ipykernel_78707/3879422107.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace = True)


    My prediction function often fails to predict the results. 
    The reason for this is that given the large number of non-repeating attributes in the COUNT columns, 
    there are a large number of attributes in the TRAIN and TEST samples that are not present in each other.

    When the COUNTY columns are removed, the accuracy increases from 25% to 88%. 
    
    !!!   Uncomments the code in the code area below to test the results.   !!!

In [33]:
df.drop(columns=['county'], axis = 1, inplace = True)
#df

/var/folders/dz/ffb7x_vj5vl9758_4vqfvj3m0000gn/T/ipykernel_78707/483174569.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['county'], axis = 1, inplace = True)


    REPORT OF MAX TREE DEPTH:
    For the above reason max tree depth is 2 because only two attributes, COUNTY and STATE, 
    are needed to divide the target values, but when these two features are removed the maximum depth of the tree increases to 4.
    Obviously the latter is more reasonable.

    REPORT OF THE NUMBER OF THE REPEATED:
    Because I have fixed the maximum depth to the number of features-1 when I designed the decision tree 
    (because the depth of the decision tree is 0 at the beginning), 
    there is no possibility of the same feature appearing at different depths, 
    given that the depth control has been implemented. So I think the REPEATED feature of the decision tree I built is 0.

    Classes are created below

In [34]:
class Tree():
    def __init__(self, TreeNode):
        self.root = TreeNode
    
    
class Edge():
    def __init__(self, df, name, value):
        self.df = df
        self.name = name
        self.value = value
        self.children = []
    
    def add_child(self, TreeNode):
        self.children.append(TreeNode)
    

class TreeNode():
    def __init__(self, df, name, depth, max_depth):
        self.df = df
        self.children = []
        self.edges = []
        self.name = name
        self.depth = depth
        self.max_depth = max_depth
    

    def add_child(self, TreeNode):
        self.children.append(TreeNode)
    
    def add_edge(self, edge):
        self.edges.append(edge)
    
    
    
    def __repr__(self, level=0):
        ret = "\t"*level+repr(self.name)+"\n"
        for child in self.children:
            ret += child.__repr__(level+1)
        return ret
    



In [35]:
# Make new treeNode
def makeTreeNode(df, feature, depth, max_depth):
    return TreeNode(df, feature, depth, max_depth)

In [36]:
# Make new subframe
def get_sub_frame(df, feature, variable):
    sub_frame = df[df[feature] == variable].reset_index(drop = True)
    return sub_frame

In [37]:
# Meke new edge
def makeEdge(Node,feature, variable):
    sub_frame = get_sub_frame(Node.df, feature, variable)
    values = sub_frame['target'].unique()
    if len(values) <= 1:
        value = values[0]
    else:
        value = None
    edge = Edge(sub_frame, variable, value)
    return edge


    Gini 


$$
\operatorname{Gini}(Y)=-\sum_{i=1}^{k}\left(P_{i}\right)^{2}
$$

In [38]:
# Input: dataframe
# Funtion: calculate the Gini impurity of the dataframe
# Output: None
# Return: float --- Gini impurity of the frame
def getTotalGini(df):
    target_values_count = df['target'].value_counts()
    num_sample = len(df)
    Gini = 0
    for i in target_values_count:
        Gini += (i/num_sample)**2
    return 1 - Gini 

In [39]:
# Input: dataframe & string---name of a feature
# Funtion: find the Gini impurity of the feature
# Output: None
# Return: float --- Gini impurity of the feature
def getSplitGini(df, feature):
    variables = df[feature].value_counts().keys().tolist()
    counts = df[feature].value_counts().tolist()
    total_num_sample = len(df)
    total_Gini = 0
    for i in range(len(variables)):
        sub_frame = get_sub_frame(df, feature, variables[i])
        sub_frame.dropna(inplace = True)
        variable_percentage = len(sub_frame)/total_num_sample
        target_values_count = sub_frame['target'].value_counts()
        num_sample = counts[i]
        part_Gini = 0
        for j in target_values_count:
            part_Gini += (j/num_sample)**2
        total_Gini += variable_percentage * part_Gini
    return 1 - total_Gini

In [40]:
# Input: TreeNode object
# Funtion: find the feature with the best information gain
# Output: None
# Return: string---name of the best feature
def find_largest_IG_Gini(Node):
    features = Node.df.keys()[:-1]
    IG_list = []
    for feature in features:
        information_gain = getTotalGini(Node.df) - getSplitGini(Node.df, feature)
        IG_list.append(information_gain)
    IG_list = np.array(IG_list)
    index_return = IG_list.argmax()
    return features[index_return]

In [41]:
# Input: TreeNode object
# Funtion: build the decision tree recursively
# Output: None
# Return: TreeNode---root of the tree
def decisionTreeGini(treeNode):
    best_feature = find_largest_IG_Gini(treeNode)
    variables = treeNode.df[best_feature].unique()
    treeNode.name = best_feature
    for variable in variables:
        edge = makeEdge(treeNode, best_feature,variable)
        treeNode.add_edge(edge)
        if edge.value != None:
            pass
        elif treeNode.depth == treeNode.max_depth:
            vars = edge.df['target'].value_counts().keys().tolist()
            counts = edge.df['target'].value_counts().tolist()
            counts = np.array(counts)
            edge.value = vars[np.argmax(counts)]
        else:
            new_tree_node = makeTreeNode(edge.df, None, treeNode.depth+1, treeNode.max_depth)
            treeNode.add_child(new_tree_node)
            edge.add_child(new_tree_node)
            decisionTreeGini(new_tree_node)
    return treeNode

    Entropy

$$
H(Y)=-\sum_{j=1}^{k} P_{j} \log P_{j}
$$

In [42]:
# Input: dataframe & string---name of a feature
# Funtion: find the entropy of the feature
# Output: None
# Return: float --- entropy of the feature
def getSplitEntropy(df, feature):
    variables = df[feature].value_counts().keys().tolist()
    counts = df[feature].value_counts().tolist()
    total_num_sample = len(df)
    total_entropy = 0
    for i in range(len(variables)):
        sub_frame = get_sub_frame(df, feature, variables[i])
        #sub_frame.dropna(inplace = True)
        variable_percentage = len(sub_frame)/total_num_sample
        target_values_count = sub_frame['target'].value_counts()
        num_sample = counts[i]
        part_entropy = 0
        for j in target_values_count:
            part_entropy += - (j/num_sample) * np.log2(j/num_sample)
        total_entropy += variable_percentage * part_entropy
    return total_entropy
        
        

In [43]:
# Input: dataframe
# Funtion: calculate the entropy of the dataframe
# Output: None
# Return: float --- entropy of the frame
def getTotalEntropy(df):
    target_values_count = df['target'].value_counts()
    num_sample = len(df)
    entropy = 0
    for i in target_values_count:
        entropy += - (i/num_sample) * np.log2(i/num_sample)
    return float(entropy)
    

In [44]:
# Input: TreeNode object
# Funtion: find the feature with the best information gain
# Output: None
# Return: string---name of the best feature
def find_largest_IG(Node):
    features = Node.df.keys()[:-1]
    IG_list = []
    for feature in features:
        information_gain = getTotalEntropy(Node.df) - getSplitEntropy(Node.df, feature)
        IG_list.append(information_gain)
    IG_list = np.array(IG_list)
    index_return = IG_list.argmax()
    return features[index_return]
    

In [45]:
# Input: TreeNode object
# Funtion: build the decision tree recursively
# Output: None
# Return: TreeNode---root of the tree
def decisionTree(treeNode):
    best_feature = find_largest_IG(treeNode)
    variables = treeNode.df[best_feature].unique()
    treeNode.name = best_feature
    for variable in variables:
        edge = makeEdge(treeNode, best_feature,variable)
        treeNode.add_edge(edge)
        if edge.value != None:
            pass
        elif treeNode.depth == treeNode.max_depth:
            vars = edge.df['target'].value_counts().keys().tolist()
            counts = edge.df['target'].value_counts().tolist()
            counts = np.array(counts)
            edge.value = vars[np.argmax(counts)]
        else:
            #print("edge.df =", edge.df.head())
            new_tree_node = makeTreeNode(edge.df, None, treeNode.depth+1, treeNode.max_depth)
            treeNode.add_child(new_tree_node)
            edge.add_child(new_tree_node)
            decisionTree(new_tree_node)
    return treeNode


In [46]:
# Input: TreeNode object & Dataframe---a row of dataframe
# Funtion: recursively predict the target value 
# Output: None
# Return: int---target value
def predictions(treeNode, sample):
    node_name = treeNode.name
    attitude = sample[node_name].values[0]
    #print('attitude = ', attitude)
    branch_val = None
    branch = None
    for i in treeNode.edges:
        if i.name == attitude:
            branch_val = i.value
            branch = i
            if branch_val != None:
                return branch_val
            else:
                branch_val = predictions(i.children[0], sample)
    return branch_val

        



In [47]:
# Put the target coloum to the last column of df
target = 'democrat'

df['target'] = df[target]
df.drop([target], axis = 1, inplace = True)

#split trainning and testing samples
df_train = df.sample(frac=0.7, random_state=1)
df_test=df.drop(df_train.index)

# Create Tree
max_depth = len(df.keys())-2
root = TreeNode(df_train, None,0,max_depth)
root2 = TreeNode(df_train, None,0,max_depth)
tree = Tree(root)

Tree = decisionTree(root)
#print(root)


/var/folders/dz/ffb7x_vj5vl9758_4vqfvj3m0000gn/T/ipykernel_78707/1605402897.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df[target]
/var/folders/dz/ffb7x_vj5vl9758_4vqfvj3m0000gn/T/ipykernel_78707/1605402897.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop([target], axis = 1, inplace = True)


In [48]:
Tree2 = decisionTreeGini(root2)

In [49]:
target_val = np.array(df_train['target'])
total_num = len(target_val)
count = 0

for i in range(len(df_train)):
    sample = df_train[i:i+1]    
    predict_val = predictions(root, sample)
    if target_val[i] == predict_val:
        count += 1
print('accuracy of training entropy tree = {:.2%}'.format(count/total_num))

target_val = np.array(df_test['target'])
total_num = len(target_val)
count = 0

for i in range(len(df_test)):
    sample = df_test[i:i+1]    
    predict_val = predictions(root, sample)
    if target_val[i] == predict_val:
        count += 1
print('accuracy of testing entropy tree = {:.2%}'.format(count/total_num))


accuracy of training entropy tree = 93.18%
accuracy of testing entropy tree = 85.35%


    The following is the prediction from the tree generated by the Gini function. 
    It can be seen that the accuracy of the tree generated by the Gini function is 26%, 
    which is slightly higher than that of the tree generated by the entropy function, 
    before removing the COUNTY and STATE features. 
    However, after removing the COUNTY and STATE features, the accuracies are exactly the same.

In [50]:
target_val = np.array(df_train['target'])
total_num = len(target_val)
count = 0

for i in range(len(df_train)):
    sample = df_train[i:i+1]    
    predict_val = predictions(root2, sample)
    if target_val[i] == predict_val:
        count += 1
print('accuracy of training Gini tree = {:.2%}'.format(count/total_num))

target_val = np.array(df_test['target'])
total_num = len(target_val)
count = 0


for i in range(len(df_test)):
    sample = df_test[i:i+1]    
    predict_val = predictions(root2, sample)
    #predict_val = 0
    #if predict_val == None:
    #    predict_val = 0
    if target_val[i] == predict_val:
        count += 1
print('accuracy of testing Gini tree = {:.2%}'.format(count/total_num))

accuracy of training Gini tree = 93.18%
accuracy of testing Gini tree = 84.93%


    By summarizing the age-related features, accuracy improved by 0.7% in the test with COUNTY and STATE removed. 
    But no improvement was seen in the unprocessed dataframe.

In [51]:
#Summarize the age-related features
df1['age'] = df1.apply((lambda x: 0 if x['age_young'] > x['age_adult'] and x['age_young'] > x['age_old'] 
else (1 if x['age_adult'] > x['age_old'] and x['age_adult'] > x['age_old'] else 2)), axis = 1)

In [52]:
# select categorical features from dataframe
df2 = df1[['democrat', 'county', 'state', 'education', 'religion', 'ethnic_male','ethnic_female','age']]

In [53]:
#drop COUNTY and STATE to increase the accuracy
df2.drop(columns=['county', 'state'], axis = 1, inplace = True)

/var/folders/dz/ffb7x_vj5vl9758_4vqfvj3m0000gn/T/ipykernel_78707/4208084113.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(columns=['county', 'state'], axis = 1, inplace = True)


In [54]:
#cleanning the data
df2.dropna(inplace = True)

/var/folders/dz/ffb7x_vj5vl9758_4vqfvj3m0000gn/T/ipykernel_78707/1018254845.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.dropna(inplace = True)


In [55]:
# Put the target coloum to the last column of df
target = 'democrat'

df2['target'] = df2[target]
df2.drop([target], axis = 1, inplace = True)

#split trainning and testing samples
df_train2 = df2.sample(frac=0.7, random_state=1)
df_test2=df2.drop(df_train.index)

# Create Tree
max_depth = len(df2.keys())-2
root3 = TreeNode(df_train2, None,0,max_depth)
#root2 = TreeNode(df_train, None,0,max_depth)
#tree = Tree(root3)

Tree = decisionTree(root3)
#Tree.levelOrder(root3)

/var/folders/dz/ffb7x_vj5vl9758_4vqfvj3m0000gn/T/ipykernel_78707/1747619249.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['target'] = df2[target]
/var/folders/dz/ffb7x_vj5vl9758_4vqfvj3m0000gn/T/ipykernel_78707/1747619249.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop([target], axis = 1, inplace = True)


In [56]:
target_val = np.array(df_train2['target'])
total_num = len(target_val)
count = 0

for i in range(len(df_train2)):
    sample = df_train2[i:i+1]
    #print('target value = ', target_val[i])
    predict_val = predictions(root3, sample)
    if target_val[i] == predict_val:
        count += 1
print('accuracy of training entropy tree= {:.2%}'.format(count/total_num))

target_val = np.array(df_test2['target'])
total_num = len(target_val)
count = 0

for i in range(len(df_test2)):
    sample = df_test2[i:i+1]
    #print('target value = ', target_val[i])
    predict_val = predictions(root3, sample)
    if target_val[i] == predict_val:
        count += 1
print('accuracy of testing entropy tree= {:.2%}'.format(count/total_num))

accuracy of training entropy tree= 89.96%
accuracy of testing entropy tree= 87.96%


Sources that helped me:   

https://www.youtube.com/watch?v=YG_nOa6-6Q8   

https://stackoverflow.com/questions/20242479/printing-a-tree-data-structure-in-python
